# **14.1 Reading_Text_Files**

Text files are everywhere — configuration files, save data, logs, Pokemon stats exported from spreadsheets. Learning to read files programmatically lets your Python code interact with the outside world. In this lesson you'll learn multiple ways to open and read text files, from loading everything at once to processing line by line, and you'll understand when each technique is appropriate for your Pokemon programs.

---

## **Creating a Sample Text File**

Before we can read files, let's create a sample file with some Pokemon data. We'll write it to disk and then demonstrate various ways to read it back.

In [ ]:
# Create a sample Pokemon team file for demonstrations
pokemon_data = """Pikachu,Electric,25
Charizard,Fire,36
Blastoise,Water,36
Venusaur,Grass,32
Gengar,Ghost,34
Snorlax,Normal,30
"""

# Write the data to a file
with open('pokemon_team.txt', 'w') as f:
    f.write(pokemon_data)

print("Created pokemon_team.txt")
print("Contents:")
print(pokemon_data)

---

## **Method 1: open() and close()**

The fundamental way to open a file is with the built-in `open()` function. You must always remember to `close()` the file when done, or you risk corrupting data and leaking system resources.

In [ ]:
# Manual open and close — you must close the file!
file = open('pokemon_team.txt', 'r')  # 'r' = read mode (default)
contents = file.read()                 # Read entire file as one string
file.close()                           # CRITICAL: always close when done

print("File contents:")
print(contents)
print(f"\nType: {type(contents)}")
print(f"Length: {len(contents)} characters")

---

## **Method 2: with Statement (Context Manager)**

The `with` statement automatically closes the file for you when the block ends — even if an exception occurs. This is the **recommended** way to work with files in modern Python because it prevents resource leaks.

In [ ]:
# BEST PRACTICE: with statement auto-closes the file
with open('pokemon_team.txt', 'r') as f:
    contents = f.read()
    # File is automatically closed when we exit this block

print("Read with 'with' statement:")
print(contents)

# The file is now closed — you can't read from f anymore
try:
    f.read()
except ValueError as e:
    print(f"\nFile is closed: {e}")

---

## **Reading Methods: read(), readline(), readlines()**

File objects have three main methods for reading content. Each is useful in different situations depending on your file size and processing needs.

In [ ]:
# Method 1: read() — entire file as one string
with open('pokemon_team.txt', 'r') as f:
    all_content = f.read()

print("read() — entire file as string:")
print(repr(all_content))  # repr shows \n newline characters
print()

# Method 2: readline() — read one line at a time
with open('pokemon_team.txt', 'r') as f:
    line1 = f.readline()  # First line (includes \n)
    line2 = f.readline()  # Second line
    line3 = f.readline()  # Third line

print("readline() — one at a time:")
print(f"Line 1: {line1.strip()}")  # .strip() removes \n
print(f"Line 2: {line2.strip()}")
print(f"Line 3: {line3.strip()}")
print()

# Method 3: readlines() — all lines as a list of strings
with open('pokemon_team.txt', 'r') as f:
    all_lines = f.readlines()  # Returns list

print("readlines() — list of lines:")
print(f"Type: {type(all_lines)}")
print(f"Number of lines: {len(all_lines)}")
print(f"First line: {all_lines[0].strip()}")

---

## **Iterating Over Lines Directly**

The most memory-efficient way to process large files is to iterate directly over the file object. Python reads one line at a time on demand, so you never load the entire file into memory at once.

In [ ]:
# BEST for large files: iterate directly (memory efficient)
print("Processing line by line:")
with open('pokemon_team.txt', 'r') as f:
    for line_num, line in enumerate(f, start=1):
        # Each line includes \n — strip it before processing
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(',')  # Split by comma
            name, ptype, level = parts
            print(f"  {line_num}. {name:12} {ptype:8} Lv.{level}")

---

## **Handling File Not Found**

Attempting to open a file that doesn't exist raises `FileNotFoundError`. Always wrap file operations in try/except when the file might not be there.

In [ ]:
# Attempt to open a file that doesn't exist
try:
    with open('missing_file.txt', 'r') as f:
        contents = f.read()
except FileNotFoundError:
    print("Error: The file does not exist!")
    print("Creating a default save file...")
    # Recover by creating a new file with default data
    with open('missing_file.txt', 'w') as f:
        f.write("Default Pokemon: Pikachu,Electric,5\n")
    print("Default file created.")

# Safe file reading function
def read_file_safely(filename: str) -> str | None:
    """Read a file, returning None if it doesn't exist."""
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Warning: '{filename}' not found")
        return None

result = read_file_safely('pokemon_team.txt')
if result:
    print(f"\nSuccessfully read {len(result)} characters")

---

## **Processing File Data into Structures**

Raw text is rarely useful on its own. Usually you want to parse it into Python data structures — lists of dictionaries, for example — so you can work with it programmatically.

In [ ]:
# Parse Pokemon file into a list of dictionaries
def load_pokemon_team(filename: str) -> list[dict]:
    """
    Read a CSV-like Pokemon file and return a list of Pokemon dicts.
    Format: Name,Type,Level
    """
    team = []
    
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:  # Skip empty lines
                    continue
                
                # Split and validate
                parts = line.split(',')
                if len(parts) != 3:
                    print(f"Warning: skipping malformed line '{line}'")
                    continue
                
                name, ptype, level_str = parts
                
                # Convert level to int, handling bad data
                try:
                    level = int(level_str)
                except ValueError:
                    print(f"Warning: invalid level '{level_str}' for {name}")
                    continue
                
                team.append({
                    'name': name,
                    'type': ptype,
                    'level': level
                })
    
    except FileNotFoundError:
        print(f"Error: '{filename}' not found")
        return []
    
    return team

# Use the function
team = load_pokemon_team('pokemon_team.txt')

print(f"Loaded {len(team)} Pokemon:\n")
for p in team:
    print(f"  {p['name']:12} ({p['type']:8}) — Level {p['level']}")

# Now we can work with the data
avg_level = sum(p['level'] for p in team) / len(team)
print(f"\nAverage level: {avg_level:.1f}")

---

## **Reading with Different Encodings**

By default, Python opens files as UTF-8 text. If you have files in other encodings (like Latin-1 or Windows-1252), you must specify the encoding explicitly or you'll get `UnicodeDecodeError`.

In [ ]:
# Create a file with special characters
special_data = """Pokémon Team:
Pikachu — Electric
Flabébé — Fairy
"""

# Write with UTF-8 (default)
with open('pokemon_utf8.txt', 'w', encoding='utf-8') as f:
    f.write(special_data)

# Read with UTF-8 (works correctly)
with open('pokemon_utf8.txt', 'r', encoding='utf-8') as f:
    contents = f.read()

print("UTF-8 file read correctly:")
print(contents)

# If you get a UnicodeDecodeError, try a different encoding:
# with open('file.txt', 'r', encoding='latin-1') as f:
#     contents = f.read()

---

## **Practice Exercises**

### **Task 1: Read Entire File**

Use `with` and `read()` to load the entire `pokemon_team.txt` file.

**Expected Output:**
```
Pikachu,Electric,25
Charizard,Fire,36
...
```

In [ ]:
# Your code here:


### **Task 2: Read First Line**

Use `readline()` to read just the first line.

**Expected Output:**
```
Pikachu,Electric,25
```

In [ ]:
# Your code here:


### **Task 3: Count Lines**

Use `readlines()` to count how many Pokemon are in the file.

**Expected Output:**
```
6 Pokemon in file
```

In [ ]:
# Your code here:


### **Task 4: Iterate and Print**

Iterate over the file and print each line with a number.

**Expected Output:**
```
1: Pikachu,Electric,25
2: Charizard,Fire,36
...
```

In [ ]:
# Your code here:


### **Task 5: Parse Into List**

Read the file and build a list of Pokemon names only.

**Expected Output:**
```
['Pikachu', 'Charizard', 'Blastoise', 'Venusaur', 'Gengar', 'Snorlax']
```

In [ ]:
# Your code here:


### **Task 6: Parse Into Dicts**

Build a list of dicts with name, type, and level keys.

**Expected Output:**
```
[{'name': 'Pikachu', 'type': 'Electric', 'level': 25}, ...]
```

In [ ]:
# Your code here:


### **Task 7: Handle Missing File**

Try to read a file that doesn't exist and catch the error.

**Expected Output:**
```
Error: File not found
```

In [ ]:
# Your code here:


### **Task 8: Filter While Reading**

Read the file and collect only Pokemon with level >= 30.

**Expected Output:**
```
['Charizard', 'Blastoise', 'Venusaur', 'Gengar', 'Snorlax']
```

In [ ]:
# Your code here:


### **Task 9: Calculate Average**

Read the file and calculate the average level of all Pokemon.

**Expected Output:**
```
Average level: 32.2
```

In [ ]:
# Your code here:


### **Task 10: Full Parser**

Write a complete function that reads the file, validates data, handles errors, and returns a list of Pokemon dicts.

**Expected Output:**
```
Successfully loaded 6 Pokemon
```

In [ ]:
# Your code here:


---

## **Summary**

- `open(filename, 'r')` opens a file for reading
- Always use `with open(...) as f:` to auto-close files
- `read()` — entire file as one string
- `readline()` — read one line at a time
- `readlines()` — all lines as a list
- **Best**: iterate directly over the file object for memory efficiency
- Each line includes `\n` — use `.strip()` to remove it
- Catch `FileNotFoundError` when files might not exist
- Specify `encoding='utf-8'` for files with special characters
- Parse text into Python structures (lists, dicts) for easy manipulation

---

## **Quick Reference**

```python
# Read entire file
with open('file.txt', 'r') as f:
    contents = f.read()  # Single string

# Read all lines into a list
with open('file.txt', 'r') as f:
    lines = f.readlines()  # List of strings

# Iterate line by line (BEST for large files)
with open('file.txt', 'r') as f:
    for line in f:
        line = line.strip()  # Remove \n
        print(line)

# Safe file reading
try:
    with open('file.txt', 'r') as f:
        data = f.read()
except FileNotFoundError:
    print("File not found")

# With encoding
with open('file.txt', 'r', encoding='utf-8') as f:
    contents = f.read()
```